<a href="https://colab.research.google.com/github/hrbolek/func2pipe/blob/news/notebooks/example01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install func2pipe

In [24]:
#import func2pipe.func2pipe as f2p
from func2pipe.func2pipe import createPipe, createSub, pipeit

## Introduction

### Unit Test

In [25]:
import unittest

class TestCase(unittest.TestCase):
  def __call__(self, *args, **kwargs):
    self.assertEqual(*args, **kwargs)
    return True

testEquality = TestCase()

### Simple Function

In [26]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@pipeit()
def add(item, amount):
  return {**item, 'result': item['value'] + amount}

expectedResult = [{'value': 0, 'result': 3}, {'value': 1, 'result': 4},
  {'value': 2, 'result': 5}]

pipe = createPipe(
    add(amount = 2),
    add(amount = 3),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': 0}, {'value': 1}, {'value': 2}]
Result: [{'value': 0, 'result': 3}, {'value': 1, 'result': 4}, {'value': 2, 'result': 5}]
Valid:  True


### Function with Yield

In [27]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@pipeit(with_yield = True)
def revealSubItem(item, itemName):
    for _ in item[itemName]:
        yield _

expectedResult = ['A', 'B', 'A', 'C', 'D', 'E']

pipe = createPipe(
    revealSubItem(itemName = 'value'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: ['A', 'B', 'A', 'C', 'D', 'E']
Valid:  True


### Function with State

In [28]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@pipeit(with_state = True)
def assignId(item, state = 0, idName = 'id'):
    return {**item, idName: state}, state + 1

expectedResult = [{'value': ['A', 'B'], 'ID': 0}, 
    {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]

pipe = createPipe(
    assignId(idName = 'ID'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: [{'value': ['A', 'B'], 'ID': 0}, {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]
Valid:  True


### Function with Yield and with State

In [29]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@pipeit(with_yield = True, with_state = True)
def assignIdToSubItem(item, state = -1, itemName = ''):
    for _ in item[itemName]:
        state = state + 1
        yield {itemName: _, 'id': state}, state

expectedResult = [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, 
  {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, 
  {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}
  ]

pipe = createPipe(
    assignIdToSubItem(itemName = 'value'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}]
Valid:  True


### Subpipe

In [30]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@pipeit()
def plus(item, amount):
  return item + amount

@pipeit()
def selectIt(item, f):
  return f(item)

expectedResult = [
  {'value': 0, 'newvalue': 2}, 
  {'value': 1, 'newvalue': 3}, 
  {'value': 2, 'newvalue': 4}
  ]

pipe = createPipe(
    createSub(assign = lambda source, result: {**source, 'newvalue': result})(
      selectIt(f = lambda item: item['value']),
      plus(amount = 2),
    ),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': 0}, {'value': 1}, {'value': 2}]
Result: [{'value': 0, 'newvalue': 2}, {'value': 1, 'newvalue': 3}, {'value': 2, 'newvalue': 4}]
Valid:  True


In [31]:
#from func2pipe import createPipe, pipeit
#import func2pipe as fp

#print(fp.func2pipe)
@pipeit()
def r(item):
  return item + 2

operator = createPipe(
    r(),
    list
)

print(operator([0, 1, 2]))

[2, 3, 4]


## Standard Operations

### Map

In [32]:
@pipeit()
def mapIt(item, f):
  return f(item)

### Filter

In [33]:
@pipeit(with_yield=True)
def filterIt(item, f):
  if f(item):
    yield item

### Reduce

In [34]:
from functools import reduce
def createReducer(f, initializer):
  def reducer(items):
    return reduce(f, items, initializer)
  return reducer

### All in One into List

In [39]:
operator = createPipe(
    filterIt(f = lambda item: item > 1),
    mapIt(f = lambda item: item + 2),
    list
)

data = [0, 1, 2, 3]
result = operator(data)
print(result)

[4, 5]


### All in One Reduced

In [40]:
operator = createPipe(
    filterIt(f = lambda item: item > 1),
    mapIt(f = lambda item: item + 2),
    createReducer(f = lambda x, y: x + y, initializer = 0)
)

data = [0, 1, 2, 3]
result = operator(data)
print(result)

9


### Combine Operators

In [41]:
operatorA = createPipe(
    filterIt(f = lambda item: item > 1)
)

operatorB = createPipe(
    mapIt(f = lambda item: item + 2)
)

operatorC = createPipe(
    operatorA,
    operatorB,
    list
)

data = [0, 1, 2, 3]
result = operatorC(data)
print(result)

[4, 5]
